In [44]:
import sys
# !{sys.executable} -m pip install altair
# !{sys.executable} -m pip install altair_data_server
# !{sys.executable} -m pip install prince
# !{sys.executable} -m pip install xgboost
# !{sys.executable} -m pip install tensorflow
# !{sys.executable} -m pip install keras
# !{sys.executable} -m pip install scikeras
# !{sys.executable} -m pip install plotly==5.5.0
!{sys.executable} -m pip install chart-studio
import pandas as pd
import numpy as np
import altair as alt
import plotly.express as px
from itertools import product
import scipy.stats as ss
alt.data_transformers.enable('data_server')

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 64 kB 996 kB/s  eta 0:00:01
  Using cached plotly-5.5.0-py2.py3-none-any.whl (26.5 MB)
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=a8e0e0b39048aeef2284282f98cad0fe98b6c11040d6abc09685da6e980272f2
  Stored in directory: /home/fangyf/.cache/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying


DataTransformerRegistry.enable('data_server')

In [31]:
mrtg_df = pd.read_csv('assets/mortgage.csv')

In [32]:
mrtg_df.head()

,id,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,...,REtype_SF_orig_time,investor_orig_time,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,hpi_orig_time,default_time,payoff_time,status_time
0,1,25,-7,25,113,41303.42,24.498336,9.2,226.29,2.899137,...,1,0,45000.0,715,69.4,9.2,87.03,0,0,0
1,1,26,-7,25,113,41061.95,24.483867,9.2,225.10,2.151365,...,1,0,45000.0,715,69.4,9.2,87.03,0,0,0
2,1,27,-7,25,113,40804.42,24.626795,9.2,222.39,2.361722,...,1,0,45000.0,715,69.4,9.2,87.03,0,0,0
3,1,28,-7,25,113,40483.89,24.735883,9.2,219.67,1.229172,...,1,0,45000.0,715,69.4,9.2,87.03,0,0,0
4,1,29,-7,25,113,40367.06,24.925476,9.2,217.37,1.692969,...,1,0,45000.0,715,69.4,9.2,87.03,0,0,0


## Check if there's missing value

Looks like there are 270 missing values in variable LTV_time, the rest data looks fine.

In [33]:
print(mrtg_df.isnull().sum())

id                           0
time                         0
orig_time                    0
first_time                   0
mat_time                     0
balance_time                 0
LTV_time                   270
interest_rate_time           0
hpi_time                     0
gdp_time                     0
uer_time                     0
REtype_CO_orig_time          0
REtype_PU_orig_time          0
REtype_SF_orig_time          0
investor_orig_time           0
balance_orig_time            0
FICO_orig_time               0
LTV_orig_time                0
Interest_Rate_orig_time      0
hpi_orig_time                0
default_time                 0
payoff_time                  0
status_time                  0
dtype: int64


In [34]:
mrtg_df[mrtg_df['LTV_time'].isnull()==True]

,id,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,...,REtype_SF_orig_time,investor_orig_time,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,hpi_orig_time,default_time,payoff_time,status_time
511121,39722,25,6,25,126,97305.61,NaN,11.350,226.29,2.899137,...,1,0,0.0,530,80.0,11.35,122.89,0,0,0
511122,39722,26,6,25,126,96673.37,NaN,11.350,225.10,2.151365,...,1,0,0.0,530,80.0,11.35,122.89,0,0,0
511123,39722,27,6,25,126,96543.78,NaN,11.350,222.39,2.361722,...,1,0,0.0,530,80.0,11.35,122.89,0,0,0
511124,39722,28,6,25,126,96411.72,NaN,11.350,219.67,1.229172,...,1,0,0.0,530,80.0,11.35,122.89,0,0,0
511125,39722,29,6,25,126,96070.49,NaN,11.350,217.37,1.692969,...,1,0,0.0,530,80.0,11.35,122.89,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619792,49658,56,-8,52,112,33090.22,NaN,8.875,181.43,1.717053,...,1,0,0.0,573,70.0,0.00,83.87,0,0,0
619793,49658,57,-8,52,112,32603.53,NaN,8.875,187.20,2.556052,...,1,0,0.0,573,70.0,0.00,83.87,0,0,0
619794,49658,58,-8,52,112,32603.53,NaN,8.875,188.37,2.868594,...,1,0,0.0,573,70.0,0.00,83.87,0,0,0
619795,49658,59,-8,52,112,32098.56,NaN,8.875,187.65,2.443648,...,1,0,0.0,573,70.0,0.00,83.87,0,0,0


In [35]:
print(pd.DataFrame({'Percentage': mrtg_df.groupby('status_time').size()*100 / len(mrtg_df)}))


             Percentage
status_time            
0             93.293536
1              2.435063
2              4.271401


In [38]:
mrtg_df.columns

Index(['id', 'time', 'orig_time', 'first_time', 'mat_time', 'balance_time',
       'LTV_time', 'interest_rate_time', 'hpi_time', 'gdp_time', 'uer_time',
       'REtype_CO_orig_time', 'REtype_PU_orig_time', 'REtype_SF_orig_time',
       'investor_orig_time', 'balance_orig_time', 'FICO_orig_time',
       'LTV_orig_time', 'Interest_Rate_orig_time', 'hpi_orig_time',
       'default_time', 'payoff_time', 'status_time'],
      dtype='object')

In [40]:
REtype_CO_levels=mrtg_df['REtype_CO_orig_time'].unique()
REtype_CO_levels.sort()

REtype_PU_levels=mrtg_df['REtype_PU_orig_time'].unique()
REtype_PU_levels.sort()

REtype_SF_levels=mrtg_df['REtype_SF_orig_time'].unique()
REtype_SF_levels.sort()

Investor_levels=mrtg_df['investor_orig_time'].unique()
Investor_levels.sort()


print('Levels of Real Estate Type Condominium: ', REtype_CO_levels)
print('Levels of Real Estate Type Planned Urban Development: ', REtype_PU_levels)
print('Levels of Real Estate Type Single-Family Home: ', REtype_SF_levels)
print('Levels of Investor: ', Investor_levels)


Levels of Real Estate Type Condominium:  [0 1]
Levels of Real Estate Type Planned Urban Development:  [0 1]
Levels of Real Estate Type Single-Family Home:  [0 1]
Levels of Investor:  [0 1]


In [43]:
fig = px.histogram(mrtg_df, x="mat_time", nbins=30, color="status_time",
                       template='simple_white',title='Distribution of Maturity time of Mortgage status')
fig.show()

ModuleNotFoundError: No module named 'plotly.validators.scatterternary'

In [ ]:
fig = px.histogram(mrtg_df, x="balance_time", nbins=30, color="status_time",
                       template='simple_white',title='Distribution of Outstanding Balance time of Mortgage status')
fig.show()

In [ ]:
fig = px.histogram(mrtg_df, x="LTV_time", nbins=30, color="status_time",
                       template='simple_white',title='Distribution of LTV of mortgage status')
fig.show()

In [ ]:
fig = px.histogram(mrtg_df, x="hpi_time", nbins=30, color="status_time",
                       template='simple_white',title='Distribution of HPI of mortgage status')
fig.show()

In [ ]:
fig = px.histogram(mrtg_df, x="FICO_orig_time", nbins=30, color="status_time",
                       template='simple_white',title='Distribution of FICO of mortgage status')
fig.show()

In [ ]:
fig = px.histogram(mrtg_df, x="LTV_orig_time", nbins=30, color="status_time",
                       template='simple_white',title='Distribution of ltv of mortgage status')
fig.show()

In [ ]:
cor_data = (mrtg_df.drop(columns=['REtype_CO_orig_time','REtype_PU_orig_time','REtype_SF_orig_time','investor_orig_time'])
              .corr().stack()
              .reset_index()     # The stacking results in an index on the correlation values, we need the index as normal columns for Altair
              .rename(columns={0: 'correlation', 'level_0': 'variable', 'level_1': 'variable2'}))
cor_data['correlation_label'] = cor_data['correlation'].map('{:.2f}'.format)  # Round to 2 decimal
cor_data.head()

In [ ]:
base = alt.Chart(cor_data).encode(
    x='variable2:O',
    y='variable:O'    
)

# Text layer with correlation labels
# Colors are for easier readability
text = base.mark_text().encode(
    text='correlation_label',
    color=alt.condition(
        alt.datum.correlation > 0.5, 
        alt.value('white'),
        alt.value('black')
    )
)

# The correlation heatmap itself
cor_plot = base.mark_rect().encode(
    color='correlation:Q'
)

cor_plot + text 

In [ ]:
dict_CO={1:'Condominium',0:'Otherwise'}
dict_PU={1: 'Planned Urban Development', 0:'Otherwise'}
dict_SF={1:'Single-Family',0:'Otherwise'}
dict_INV={1:'Borrower',0:'Otherwise'}

In [ ]:
mrtg_df = mrtg_df[['REtype_CO_orig_time','REtype_PU_orig_time','REtype_SF_orig_time','investor_orig_time']]
mrtg_df=mrtg_df.replace({"REtype_CO_orig_time": dict_CO})
mrtg_df=mrtg_df.replace({"REtype_PU_orig_time": dict_PU})
mrtg_df=mrtg_df.replace({"REtype_SF_orig_time": dict_SF})
mrtg_df=mrtg_df.replace({"investor_orig_time": dict_INV})

In [ ]:
at_var1 = ('REtype_CO_orig_time','REtype_PU_orig_time','REtype_SF_orig_time','investor_orig_time')
cat_var_prod = list(product(cat_var1,cat_var1, repeat = 1))

In [ ]:
result = []
for i in cat_var_prod:
    if i[0] != i[1]:
        result.append((i[0],i[1],list(ss.chi2_contingency(pd.crosstab(
                            mrtg_df[i[0]], mrtg_df[i[1]])))[1]))

In [ ]:
pd.options.display.float_format = '{:20,.10f}'.format
chi_test_output = pd.DataFrame(result, columns = ['var1', 'var2', 
                                                       'coeff'])
## Using pivot function to convert the above DataFrame into a crosstab
chi_test_output.pivot(index='var1', columns='var2', values='coeff')